In [2]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Initialize batch size, image size and input shape
batch_size = 32
image_size = (299, 299) # Define the size of images
input_shape = image_size + (3,) # Add channel dimension to image size for RGB
keras = tf.keras 

# Image data generator for data augmentation
img_data_gen = ImageDataGenerator(
    rescale=1./255, # Rescale pixel values from 0-255 to 0-1
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Creating training dataset
train_ds = img_data_gen.flow_from_directory(
    '../PotatoImagesData/Train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=123
)
classes = list(train_ds.class_indices.keys())

# Creating validation dataset
val_ds = img_data_gen.flow_from_directory(
    '../PotatoImagesData/Valid',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=123
)

# Creating test dataset
test_ds = img_data_gen.flow_from_directory(
    '../PotatoImagesData/Test',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=123
)
train_ds.class_indices

Found 900 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}

In [2]:
# Initialize the base model (InceptionV3)
base_model = InceptionV3(
    weights="imagenet",
    include_top=False,
    input_shape=input_shape
)

# Freeze the layers of the base model
base_model.trainable = False

In [3]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x) # Add a global average pooling layer
x = Dense(1024, activation='relu')(x)  # New FC layer, you can adjust the number of neurons
predictions = Dense(train_ds.num_classes, activation='softmax')(x)  # Final softmax layer for classification

# Define the new model
model = Model(inputs=base_model.input, outputs=predictions)

In [4]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001), # Optimizer
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), # Loss function
    metrics=['accuracy']
)

# Early stopping callback to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
evaluation_results = model.evaluate(test_ds)
print(f"Test Loss: {evaluation_results[0]}, Test Accuracy: {evaluation_results[1]}")

# Save the model
model.save('inceptionv3.keras')

Epoch 1/50
29/29 [==============================] - 130s 4s/step - loss: 0.7162 - accuracy: 0.6733 - val_loss: 0.4755 - val_accuracy: 0.8433
Epoch 2/50
29/29 [==============================] - 126s 4s/step - loss: 0.4017 - accuracy: 0.8611 - val_loss: 0.4077 - val_accuracy: 0.8567
Epoch 3/50
29/29 [==============================] - 147s 5s/step - loss: 0.3106 - accuracy: 0.8922 - val_loss: 0.3114 - val_accuracy: 0.8667
Epoch 4/50
29/29 [==============================] - 124s 4s/step - loss: 0.2620 - accuracy: 0.9156 - val_loss: 0.2985 - val_accuracy: 0.9000
Epoch 5/50
29/29 [==============================] - 126s 4s/step - loss: 0.2351 - accuracy: 0.9267 - val_loss: 0.2565 - val_accuracy: 0.9067
Epoch 6/50
29/29 [==============================] - 134s 5s/step - loss: 0.2049 - accuracy: 0.9367 - val_loss: 0.3055 - val_accuracy: 0.8900
Epoch 7/50
29/29 [==============================] - 123s 4s/step - loss: 0.1978 - accuracy: 0.9389 - val_loss: 0.2217 - val_accuracy: 0.9200
Epoch 8/50
29

In [4]:
model = tf.keras.models.load_model("inceptionv3.keras")

In [7]:
import tensorflow as tf
import numpy as np

# Utility function for preprocessing an image
def get_img_array(image_path, target_size=image_size):
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img /= 255. # Normalize pixel values
    return img

# Make predictions on sample images
predictions = model.predict(np.array([
    get_img_array("../PotatoImagesData/Train/Potato___Early_blight/bb07a1b7-b9ad-4016-a013-9ff0e4636d4a___RS_Early.B 7156.JPG"),
    get_img_array("../PotatoImagesData/Train/Potato___Late_blight/fd35fb86-6558-40f2-b8ac-6922e2107af5___RS_LB 4095.JPG"),
    get_img_array("../PotatoImagesData/Train/Potato___healthy/Potato_healthy-103-_0_3577.jpg"),
]))

# Print predicted classes for each image
for i in predictions:
    print(classes[np.argmax(i)]) # Print the class with the highest predicted probability

1/1 [==============================] - 0s 169ms/step
Potato___Early_blight
Potato___Late_blight
Potato___healthy
